In [34]:

# Клонируем репозиторий с данными
!git clone https://github.com/nsuemdm-lab/sber_hack_data.git
%cd sber_hack_data
# Проверяем, что файлы на месте
!ls

Cloning into 'sber_hack_data'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 12 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (12/12), 19.76 MiB | 36.46 MiB/s, done.
Resolving deltas: 100% (1/1), done.
/content/sber_hack_data/sber_hack_data/sber_hack_data
connection.parquet   geoBoundaries-RUS-ADM0-all.zip  market_access.parquet
consumption.parquet  LICENSE


In [45]:
# Устанавливаем библиотеки. Ключ -q для "тихой" установки без лишнего вывода
!pip install geopandas -q

In [36]:
!ls *.zip

geoBoundaries-RUS-ADM0-all.zip


In [37]:
!mkdir shape

In [38]:
# Распаковываем архив в созданную директорию
!unzip -q geoBoundaries-RUS-ADM0-all.zip -d shape/

In [39]:
import pandas as pd
import geopandas as gpd
import folium
print("Библиотеки импортированы.")

Библиотеки импортированы.


In [65]:
!ls -a shape/

.
..
CITATION-AND-USE-geoBoundaries.txt
geoBoundaries-RUS-ADM0.dbf
geoBoundaries-RUS-ADM0.geojson
geoBoundaries-RUS-ADM0-metaData.json
geoBoundaries-RUS-ADM0-metaData.txt
geoBoundaries-RUS-ADM0-PREVIEW.png
geoBoundaries-RUS-ADM0.prj
geoBoundaries-RUS-ADM0.shp
geoBoundaries-RUS-ADM0.shx
geoBoundaries-RUS-ADM0_simplified.dbf
geoBoundaries-RUS-ADM0_simplified.geojson
geoBoundaries-RUS-ADM0_simplified.prj
geoBoundaries-RUS-ADM0_simplified.shp
geoBoundaries-RUS-ADM0_simplified.shx
geoBoundaries-RUS-ADM0_simplified.topojson
geoBoundaries-RUS-ADM0.topojson
mo.shp


In [67]:
# Пути к файлам в окружении Colab
CONSUMPTION_DATA_PATH = 'consumption.parquet'
SHAPEFILE_PATH = 'shape/geoBoundaries-RUS-ADM0.shp'

In [69]:
# Загружаем данные СберИндекс
df_consumption = pd.read_parquet(CONSUMPTION_DATA_PATH)
print("Данные о расходах:")
print(df_consumption.head())
# Загружаем геоданные
gdf_municipalities = gpd.read_file(SHAPEFILE_PATH)
print("\nГеоданные:")
print(gdf_municipalities.head())

Данные о расходах:
      date  territory_id              category  value
0  2023-01             1        Продовольствие   7692
1  2023-01             1              Здоровье   1271
2  2023-01             1          Маркетплейсы   2505
3  2023-01             1  Общественное питание   1142
5  2023-01             1             Транспорт   1718

Геоданные:
            shapeName shapeISO                  shapeID shapeGroup shapeType  \
0  Russian Federation      RUS  89588193B69022144921215        RUS      ADM0   

                                            geometry  
0  MULTIPOLYGON (((130.82214 42.4629, 130.82209 4...  


In [70]:
# 1. Фильтруем только категорию 'Все категории'
df_total_spend = df_consumption[df_consumption['category'] == 'Все категории'].copy()
# 2. Группируем по ID территории и вычисляем среднее
df_agg = df_total_spend.groupby('territory_id')['value'].mean().reset_index()
# 3. Переименовываем колонку для ясности
df_agg.rename(columns={'value': 'avg_consumption'}, inplace=True)
print("Агрегированные данные:")
print(df_agg.head())

Агрегированные данные:
   territory_id  avg_consumption
0             1     27129.333333
1             2     20444.125000
2             3     20878.125000
3             4     22012.875000
4             5     20778.833333


In [101]:
# ВАЖНО: Имена колонок-ключей должны совпадать.
# В шейп-файле это 'oktmo', в данных Сбера - 'territory_id'.
# Также важен тип данных. Приведем оба ключа к строковому типу.
gdf_municipalities['shapeID'] = gdf_municipalities['shapeID'].astype(str)
df_agg['territory_id'] = df_agg['territory_id'].astype(str)
# Объединяем таблицы
merged_gdf = gdf_municipalities.merge(
df_agg,
left_on='shapeID',
right_on='territory_id',
how='left' # 'left' сохраняет все полигоны, даже если для них нет данных

)
# Проверка результата
matched_rows = merged_gdf['shapeID'].notna().sum()
total_rows = len(merged_gdf)
print(f"Объединение завершено. Найдено соответствий для {matched_rows} из {total_rows} муниципалитетов.")
assert matched_rows > 0, "Не найдено ни одного соответствия. Проверьте имена и типы ключей!"

Объединение завершено. Найдено соответствий для 1 из 1 муниципалитетов.
1


In [105]:
# Создаем базовую карту с центром в РФ
m = folium.Map(location=[61.52, 105.31], zoom_start=3)

# Добавляем слой с географическими границами
# Убираем муниципалитеты, для которых нет данных, чтобы не загромождать легенду
# plot_gdf = merged_gdf.dropna(subset=['avg_consumption']) # Эта строка больше не нужна для базовой карты

# Используем объединенный gdf, который содержит географические границы
# Хотя он не содержит данных о потреблении для большинства полигонов
folium.GeoJson(
    merged_gdf, # Используем merged_gdf, который содержит геометрию
    name='Географические границы'
).add_to(m)

# Добавляем управление слоями
folium.LayerControl().add_to(m)

# Отображаем карту
m


Output hidden; open in https://colab.research.google.com to view.